In [3]:
!pip install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 43.3 MB/s eta 0:00:00
  Attempting uninstall: mock
    Found existing installation: mock 5.1.0
    Uninstalling mock-5.1.0:
      Successfully uninstalled mock-5.1.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.227.0
    Uninstalling sagemaker-2.227.0:
      Successfully uninstalled sagemaker-2.227.0


In [5]:
import sagemaker
import boto3

sess = sagemaker.Session()

sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

## Role management
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

session = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn:{role}")
print(f"sagemaker session region:{sess.boto_region_name}")

sagemaker role arn:arn:aws:iam::901822506991:role/service-role/AmazonSageMaker-ExecutionRole-20240906T224811
sagemaker session region:us-east-1


In [6]:
from sagemaker.huggingface.model import HuggingFaceModel

## getting hub Model
hub = {
    "HF_MODEL_ID": "distilbert-base-uncased-distilled-squad",
    "HF_TASK": "question-answering"
}

## create huggingface model class
huggingface_model = HuggingFaceModel(
    env=hub,
    role=role,
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39"
)

## deploy model
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge"
)

## example
data = {
    "inputs": {
        "question": "what purav do?",
        "context": "My name is Purav and I live in India. I am doing Freelancing in Data Science"
    }
}

## predict
predictor.predict(data)

------!

{'score': 0.5432309508323669,
 'start': 49,
 'end': 76,
 'answer': 'Freelancing in Data Science'}

In [7]:
## example
data = {
    "inputs": {
        "question": "what John do?",
        "context": "My name is John and I working at NGO"
    }
}

## predict
predictor.predict(data)

{'score': 0.45156461000442505,
 'start': 22,
 'end': 36,
 'answer': 'working at NGO'}